In [5]:
from csat import locator
import os
from matplotlib.pyplot import imread
import numpy as np
from osgeo import gdal, osr
import cartopy.crs as ccrs


def readin(granule_name, band):
    filename = locator.search(
        'Landsat', 'L1', name=granule_name, band=str(band))[0]
    return imread(filename)


def readin_metadata(granule_name):
    def gen_metadata_dir(fobj):
        odata = {}
        for fline in fobj:
            if fline.strip() == 'END':
                return odata
            line = [l.strip() for l in fline.strip().split('=')]
            if line[0] == 'GROUP':
                odata[line[1]] = gen_metadata_dir(fobj)
            elif line[0] == 'END_GROUP':
                return odata
            else:
                odata[line[0]] = line[1].replace('"', '')

    f = open(locator.search('Landsat', 'L1meta', name=granule_name)[0])
    output_data = gen_metadata_dir(f)
    f.close()
    return output_data['L1_METADATA_FILE']


def granules_available():
    return list(set([x.split('/')[-2] for x in
                     locator.search('Landsat', 'L1', name='*', band='*')]))

# The counts to radiance/reflectance/temperature calculations are from
# http://landsat.usgs.gov/Landsat8_Using_Product.php


def counts_to_radiance(counts, granule_name, band):
    metdata = readin_metadata(granule_name)['RADIOMETRIC_RESCALING']
    band = str(int(band))
    if int(band) < 10:
        mult = float(metdata['RADIANCE_MULT_BAND_' + band])
    else:
        mult = 0.0003342
    adder = float(metdata['RADIANCE_ADD_BAND_' + band])
    # print mult, adder
    return (mult * counts) + adder


def counts_to_reflectance(counts, granule_name, band):
    if int(band) > 9:
        raise ValueError('No reflectance for IR bands')
    metdata = readin_metadata(granule_name)
    sun_elev = np.sin(
        np.pi * float(metdata['IMAGE_ATTRIBUTES']['SUN_ELEVATION']) / 180.)
    metdata = metdata['RADIOMETRIC_RESCALING']
    band = str(int(band))
    return ((float(metdata['REFLECTANCE_MULT_BAND_' + band]) * counts) + float(metdata['REFLECTANCE_ADD_BAND_' + band])) / sun_elev


def counts_to_btemp(counts, granule_name, band):
    if int(band) < 10:
        raise ValueError('BT calc ony for IR bands')
    band = str(int(band))
    metdata = readin_metadata(granule_name)['TIRS_THERMAL_CONSTANTS']
    k1 = float(metdata['K1_CONSTANT_BAND_' + band])
    k2 = float(metdata['K2_CONSTANT_BAND_' + band])
    radiance = counts_to_radiance(counts, granule_name, band)
    # print k1, k2
    btemp = k2 / np.log((k1 / radiance) + 1)
    return np.where(btemp > 200, btemp, np.nan)

# Geolocation code from
# https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file


def geolocate(granule_name, x, y=None, band=1):
    '''Return the location of a specific pixel 
    x = i0, y = j0
    or for a list of pixels 
    x = [(i0, j0), (i1, j1), ...] for the granule
    specified by granule_name

    The band can be changed if band 1 doesn't exist or if the pan
    band is desired.

    Returns ValueError if that granule doesn't exist'''
    try:
        filename = locator.search('Landsat', 'L1', name=granule_name,
                                  band=str(band))[0]
    except:
        raise(ValueError, 'No files for specified granule')

    ds = gdal.Open(filename)
    old_cs = osr.SpatialReference()
    old_cs.ImportFromWkt(ds.GetProjectionRef())

    wgs84_wkt = """
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.01745329251994328,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]]"""
    new_cs = osr.SpatialReference()
    new_cs.ImportFromWkt(wgs84_wkt)

    # Create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs, new_cs)

    # Get the details of the landsat grid
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()

    def landsat_coords(i, j):
        x = gt[0] + i*gt[1] + j*gt[2]
        y = gt[3] + (width-i)*gt[4] + (height-j)*gt[5]
        return x, y

    if y is not None:  # Only a single point
        return transform.TransformPoint(*landsat_coords(x, y))

    else:  # Lots of points
        return transform.TransformPoints(
            list(map(landsat_coords, *x)))


def bulk_metadata(fname):
    data = np.genfromtxt(fname,
                         dtype=[('sceneID', 'S21'),
                                ('sensor', 'S8'),
                                ('acquisitionDate', 'S10'),
                                ('dateUpdated', 'S10'),
                                ('browseAvailable', 'S1'),
                                ('browseURL', 'S85'),
                                ('path', 'i'),
                                ('row', 'i'),
                                ('upperLeftCornerLatitude', 'f'),
                                ('upperLeftCornerLongitude', 'f'),
                                ('upperRightCornerLatitude', 'f'),
                                ('upperRightCornerLongitude', 'f'),
                                ('lowerLeftCornerLatitude', 'f'),
                                ('lowerLeftCornerLongitude', 'f'),
                                ('lowerRightCornerLatitude', 'f'),
                                ('lowerRightCornerLongitude', 'f'),
                                ('sceneCenterLatitude', 'f'),
                                ('sceneCenterLongitude', 'f'),
                                ('cloudCover', 'f'),
                                ('cloudCoverFull', 'f'),
                                ('dayOrNight', 'S1'),
                                ('sunElevation', 'f'),
                                ('sunAzimuth', 'f'),
                                ('receivingStation', 'S3'),
                                ('sceneStartTime', 'S25'),
                                ('sceneStopTime', 'S25'),
                                ('DATA_TYPE_L1', 'S4'),
                                ('cartURL', 'S96')],
                         skip_header=1,
                         delimiter=',',
                         usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                  10, 11, 12, 13, 14, 15, 16,
                                  17, 18, 19, 24, 26, 27, 28,
                                  29, 30, 53, 54])
    return data


ModuleNotFoundError: No module named 'csat'

In [3]:
import gdal

ds = gdal.Open("C:/Users/kathe/OneDrive - Imperial College London/MSci Project/sentinel2A_20210928.tif")
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 

#gdalinfo ~/somedir/somefile.tif 

ModuleNotFoundError: No module named 'gdal'